In [1]:
import sys

from util.django import restart_django

sys.path.append("../../src")

restart_django(imprimir_versoes=True)

Python 3.12.1 (tags/v3.12.1:2305ca5, Dec  7 2023, 22:03:25) [MSC v.1937 64 bit (AMD64)] on win32
Django 5.0.4


Esse notebook tem como objetivo exemplificar o uso das classes de pessoas e usuários do app common

In [2]:
from common.models import *
from datetime import date
import pandas as pd
from django.core.exceptions import ValidationError

## Pessoa

Ao chamar o `save` de qualquer subclasse de pessoa `full_clean` é chamado, de forma que os campos sempre são validados antes de serem salvos. Nesse caso o príncipal campo sendo válidado é o `cpf`.

Todos os campos de pessoa física são apresentados nesse exemplo.

In [3]:
bindo = PessoaFisica(cpf='11484174510', nome='Eduardo', sobrenome='C. de Souza', data_nascimento=date(2006, 7, 15))
try:
    bindo.save()
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo não foi salvo pois: {'codigo': [ValidationError(['CPF inválido'])]}


In [4]:
bindo = PessoaFisica(cpf='59247136170812', nome='Eduardo', sobrenome='C. de Souza', data_nascimento=date(2006, 7, 15))
try:
    bindo.save()
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo não foi salvo pois: {'__all__': [ValidationError(['O codigo de PessoaFisica deve ser um cpf'])]}


In [5]:
bindo = PessoaFisica(cpf='11484174518', nome='Eduardo', sobrenome='C. de Souza', data_nascimento=date(2006, 7, 15))
try:
    bindo.save()
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo foi salvo no banco de dados


Também podemos usar o método `create` para criar uma pessoa - naturalmente `full_clean` continua sendo chamado antes de salvar.

In [6]:
PessoaFisica.pessoas.create(cpf='66666666666', nome='Coisa', sobrenome='Ruim', data_nascimento=date(2006, 6, 6))

66666666666

Podemos resgatar `bindo` por meio da chave primária de `Pessoa`, por consequência também chave primária das subclasses de `Pessoa`, `codigo`. Todavia para `PessoaFisica` podemos chamá-la por `cpf` na criação e na edição `cpf` sempre é um campo annotado, além disso `cpf` possuí métodos getter e setter que modificam o `codigo`.

In [7]:
pd.DataFrame(PessoaFisica.pessoas.simple().filter(cpf='11484174518'))

,nome,sobrenome,data_nascimento,cpf
0,Eduardo,C. de Souza,2006-07-15,11484174518


In [8]:
pd.DataFrame(PessoaFisica.pessoas.filter(cpf='11484174518').values())

,codigo,pessoa_id,nome,sobrenome,data_nascimento,cpf
0,11484174518,11484174518,Eduardo,C. de Souza,2006-07-15,11484174518


Note que o método `simple` pode ser util, mas não retorna a instância de pessoa física, além de perder outros campos de relacionamentos. Talvez seja, em alguns casos em que a eficiência deve ser priorizada, utilizar o método `only`.

In [9]:
isinstance(PessoaFisica.pessoas.simple().filter(cpf='11484174518')[0], dict)

True

In [10]:
isinstance(PessoaFisica.pessoas.filter(cpf='11484174518')[0], PessoaFisica)

True

Note que `codigo` é `editable=False`, logo não deve ser alterado, embora exista essa possibilidade. Caso altere `codigo` outra linha de `Pessoa` será criada, isso ocasionará erros imprevisíveis e assustadores.

Trabalhar com `PessoaJuridica` é análogo a trabalhar com `PessoaFisica`, exceto que o codinome de `codigo` é `cnpj` e não possuí o campo `data_nascimento`, obviamente a verificação do `codigo` verifica se é um `cnpj` válido. Portanto, não será detalhado.

## UsuarioGenerico

O `UsuarioGenerico` estabelece uma relação com `PessoaUsuario` que define o escopo do usuário. De forma simplória, uma `Pessoa` pode estar associada a vários usuários, enquanto um usuário está associado a somente uma pessoa, pois o usuário pode existir em diferentes escopos sob mesmo `codigo`, ou seja, sendo a mesma `Pessoa`. No entanto, não é recomendado usar `UsuarioGenerico` em sua forma pura, mas sim com um proxy. O usuário que é uma pessoa jurídica deve usar `UsuarioGenericoPessoJuridica` e o que é uma pessoa física `UsuarioGenericoPessoaFisica`. Esses proxys visam simular a herança de pessoa, embora ela não exista, de modo a simplificar o trabalho.

### Da criação de novos usuários

A criação de novos usuários será pelas classes proxy de `UsuarioGenericoPessoaFisica` ou `UsuarioGenericoPessoaJuridica`, é altamente recomendado o uso do método `criar_usuario` e `criar_superusuario`, alternativamente é possível usar o builder. Isso quando feito em código, quando usando o form, deve herdar dos forms que serão feitos no common.

Tanto no `criar_usuario` como como será no form, a senha é haseada e os dados inseridos validados. Portanto, não utilize métodos alternativos e mandrakes para a criação de usuários, pois eles podem não ser validados ou salvar a senha bruta, ambos seriam péssimos.

In [11]:
escocesa_ltda = UsuarioGenericoPessoaJuridica.usuarios.criar_usuario(
    cnpj='12911507000186',
    password='espelho',
    email='mr@gmail.com',
    telefone='31984735439',
    razao_social='Escocia LTDA',
    nome_fantasia='Escocesa Especular'
)

Ou

In [12]:
builder = UsuarioGenericoPessoaJuridica.usuarios.create_builder()
builder.cnpj = '43694255431512'
builder.password = 'bingos'
builder.email = 'algum@gmail.com'
builder.telefone = '32984735439'
builder.razao_social = 'Bingos Dingos LTDA'
builder.nome_fantasia = 'Bingos o Dingo'
bingos_dingos = builder.build_usuario()

Criando um escopo para um grupo de usuários

In [13]:
from scope_auth.models import Scope

In [14]:
outro_escopo = Scope.scopes.create()

In [15]:
bindo = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
    cpf='51283834235',
    scope=outro_escopo,
    password='bingosdingos@1234',
    email='bindo@gmail.com',
    telefone='32999781209',
    nome='Bindo',
    sobrenome='Dingo'
)

Alterando o usuário `bindo`, modificando os dados de pessoa e os dados de usuário utilizando o proxy

In [16]:
bindo.nome = 'Bingo'
bindo.email = 'dem@hotmail.com'
bindo.save()

Somente criando outro usuário em mais um escopo

In [17]:
mais_um_escopo = Scope.scopes.create()

coringa = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
    cpf='37705858797',
    scope=mais_um_escopo,
    password='senha$#$doiasdf2345',
    email='coringa@gmail.com',
    telefone='32999781209',
    nome='Coringa',
    sobrenome=', O Brincalhão'
)

### Resgatando usuários do banco de dados

Ao resgatar usuários do banco de dados nem sempre será possível recuperá-lo como instância de um dos `proxy`, ou não será possível recuperar um queryset dessa maneira. Portanto, mostraremos como resgatar os dados pelos `proxy` e como realizar a coerção de um queryset da classe base ou de uma instância da classe base para os proxys.

É importante saber que os proxys já filtram pelos usuários de pessoa física e de pessoa jurídica.

Os campos de pessoa física e pessoa jurídica são anotados no queryset logo após ser criado, os campos de scope e pessoa são anotados nas superclasses. Também são fornecidas properties e setters para membros de pessoa. Os métodos de `clean` e `save` chamam tambem os de pessoa e pessoa_usuario, conforme em:

```python
bindo.nome = 'Bingo'
bindo.email = 'dem@hotmail.com'
bindo.save()
```

In [18]:
pd.DataFrame(UsuarioGenericoPessoaJuridica.usuarios.simple())

,telefone,email,scope,razao_social,nome_fantasia,cnpj
0,32999781209,sistemalojasaas@gmail.com,1,Sistema Loja SaaS CEFET INF,Sistema Loja SaaS,97456911807888
1,None,bilu@gmail.com,1,Bilu LTDA,Bilu vendass,34561499177486
2,31984735439,mr@gmail.com,1,Escocia LTDA,Escocesa Especular,12911507000186
3,32984735439,algum@gmail.com,1,Bingos Dingos LTDA,Bingos o Dingo,43694255431512


Filtrando por escopo

In [19]:
pd.DataFrame(UsuarioGenericoPessoa.usuarios.filter(scope=outro_escopo).values())

,id,password,last_login,is_superuser,is_staff,is_active,date_joined,pessoa_usuario_id,telefone,email,codigo,scope,pessoa
0,5,pbkdf2_sha256$720000$T4TGZWhzyTaVSeRJX3aLZ3$wd...,None,False,False,True,2024-05-30 15:38:01.125103+00:00,6,32999781209,dem@hotmail.com,51283834235,3,51283834235


Pegando uma instância por `cnpj` e `scope`

In [20]:
UsuarioGenericoPessoaJuridica.usuarios.get(cnpj='43694255431512', scope=Scope.scopes.default_scope()).__dict__

{'_state': <django.db.models.base.ModelState at 0x283875226f0>,
 'id': 4,
 'password': 'pbkdf2_sha256$720000$jMeYtGrV3oF0E4GCKiwucC$1CkoOgoAI5LoEyFKKSt2ZN80ryRucgpj+DVpdGpyDHY=',
 'last_login': None,
 'is_superuser': False,
 'is_staff': False,
 'is_active': True,
 'date_joined': datetime.datetime(2024, 5, 30, 15, 38, 0, 758007, tzinfo=datetime.timezone.utc),
 'pessoa_usuario_id': 5,
 'telefone': '32984735439',
 'email': 'algum@gmail.com'}

Imagine você tem um queryset de `UsuarioGenerico` mas não te interessam as pessoas jurídicas, somente as físicas. Todavia, para punir os que tentaram burlar o sistema, você quer saber quais são as pessoas jurídicas que estavam no queryset e não deveriam estar ali.

In [21]:
qs = UsuarioGenerico.usuarios.all()

In [22]:
pessoas_fisicas = qs.as_usuarios_pessoa_fisica()
pd.DataFrame(pessoas_fisicas.simple())

,telefone,email,scope,nome,sobrenome,data_nascimento,cpf
0,32999781209,dem@hotmail.com,3,Bingo,Dingo,None,51283834235
1,32999781209,coringa@gmail.com,4,Coringa,", O Brincalhão",None,37705858797


In [23]:
pessoas_juridicas = qs.as_usuarios_pessoa_juridica()
pd.DataFrame(pessoas_juridicas.simple())

,telefone,email,scope,razao_social,nome_fantasia,cnpj
0,32999781209,sistemalojasaas@gmail.com,1,Sistema Loja SaaS CEFET INF,Sistema Loja SaaS,97456911807888
1,None,bilu@gmail.com,1,Bilu LTDA,Bilu vendass,34561499177486
2,31984735439,mr@gmail.com,1,Escocia LTDA,Escocesa Especular,12911507000186
3,32984735439,algum@gmail.com,1,Bingos Dingos LTDA,Bingos o Dingo,43694255431512


Alternativamente

In [24]:
pessoas_fisicas = UsuarioGenericoPessoaFisica.usuarios.from_usuarios_queryset(qs)
pd.DataFrame(pessoas_fisicas.simple())

,telefone,email,scope,nome,sobrenome,data_nascimento,cpf
0,32999781209,dem@hotmail.com,3,Bingo,Dingo,None,51283834235
1,32999781209,coringa@gmail.com,4,Coringa,", O Brincalhão",None,37705858797


In [25]:
try:
    print(qs[0].as_usuario_pessoa_fisica())
except TypeError as e:
    print(e)

O usuário não pode ser convertido pois a pessoa 97456911807888 não é uma pessoa física.


In [26]:
try:
    print(qs[0].as_usuario_pessoa_juridica())
except TypeError as e:
    print(e)

<UsuarioGenerico: {codigo=97456911807888, scope=Scope object (1)}>


Basicamente isso exemplifica como trabalhar com um usuário. Em caso de herança o manager e queryset também deverão ser herdados.